In [11]:
import sys
sys.path.append('../modules')
import likelihood_predictor
from likelihood_predictor import PlastPredictor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.stats import zscore
import pickle
from sklearn.feature_selection import SelectFromModel

In [12]:
pl_full = pd.read_pickle('../database/plasticizer_data_v10_polarity.pkl')
pl_pol = pd.concat([pl_full[pl_full.columns[1:195]], pl_full['Polarity']], axis=1)
all_cols = pl_pol.columns.to_numpy()
pl_data = pl_pol[all_cols].to_numpy()
lin_data = pd.read_pickle('../database/linolein_test.pkl')
lin_data['Polarity'] = 0.048856
lin_data = lin_data[all_cols].to_numpy()
org_full = pd.read_pickle('../database/org_v3.pkl')

In [13]:
psim1 = open("pubs_similarity.txt", 'r')
psim11 = [line.rstrip('\n') for line in psim1]
psim2 = open("pubs_othersim.txt", 'r')
psim22 = [line.rstrip('\n') for line in psim2]

In [14]:
org_full
# org_full['Dsim'] = psim11
# org_full['Nasim'] = psim22
org_full = org_full.sort_values(by ='DEHP_sim')

In [15]:
org_full = org_full[:5000]
org_data = org_full[all_cols].to_numpy()

In [16]:
reg_param = 10
pp = PlastPredictor(reg_param)
pp_model = pp.fit_model(pl_data, org_data)


In [17]:
org_acc = pp.predict(org_data, type='binary', class_id='neg')
pl_acc = pp.predict(pl_data, type='binary', class_id='pos')

In [18]:
org_acc, pl_acc

(0.9802, 0.9238095238095239)

In [19]:
cc=pp.clf.coef_

In [20]:
np.count_nonzero(cc)

105

In [21]:
clf_file = 'clf_oneiteration.pkl'
scaler_file = 'scaler_oneiteration.pkl'
pp.save_model(clf_file, scaler_file)

In [24]:
compar = cc[0] != 0
compar = compar.tolist()

In [25]:
pl_temp = pl_data.tolist()
org_temp = org_data.tolist()

In [26]:
pl_list = []

for i in range(0,210):
    count = 0
    p1 = []
    for j in compar:
        #print(j)
        if j:
#             print(type(j))
            p1.append(pl_temp[i][count])
        else:
            p1.append(False)
            
        count += 1

    pl_list.append(p1)
    

In [27]:
org_list = []

for i in range(0, 5000):
    count = 0
    o1 = []
    for j in compar:
        if j:
            o1.append(org_temp[i][count]) 
        else:
            o1.append(False)
           
        count += 1
            
    org_list.append(o1)


In [31]:
df1=pd.DataFrame(data=pl_list[0])

In [33]:
df1.to_excel('features_oneiteration.xls')